## Q. 기상 실측 데이터를 토대로 태양광 발전량인 amount(2023-10-15) 값을 예측해보시오.

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from torch.utils.data import TensorDataset, DataLoader

pd.set_option('display.max_column', 20)
pd.set_option('display.max_row', 100)

data = pd.read_csv('/content/energy_tobigs_question.csv')
data['time'] = pd.to_datetime(data['time'])

# 피처와 타겟 변수 설정
features = ['cloud', 'temp', 'humidity', 'ground_press', 'wind_speed', 'wind_dir', 'rain', 'snow', 'dew_point', 'vis', 'uv_idx', 'azimuth', 'elevation']
target = 'amount'

# 데이터셋 분리 (2023-10-14까지 학습, 2023-10-15 하루 예측)
train_data = data[data['time'] < '2023-10-15'].copy()
test_data = data[(data['time'] >= '2023-10-15') & (data['time'] < '2023-10-16')].copy()

# 학습 데이터에 NaN 값 제거
train_data = train_data.dropna()

# 데이터 스케일링
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

train_data.loc[:, features] = scaler_x.fit_transform(train_data[features])
train_data.loc[:, target] = scaler_y.fit_transform(train_data[[target]])

test_data.loc[:, features] = scaler_x.transform(test_data[features])

### 시퀀스 데이터 생성 함수 작성


In [5]:
# 텐서로 변환해서 텐서데이터셋을 생성하세요. (1~6 번에 내용을 채워보세요.)

# 시퀀스 데이터 생성 함수
# dataX: 시퀀스 데이터를 담은 리스트 (예: [[시퀀스1], [시퀀스2], ...])
# dataY: 각 시퀀스에 대응하는 타겟 값 (예: 발전량)

def build_dataset(time_series, seq_length):
    dataX, dataY = [], []
    for i in range(len(time_series) - seq_length):
        dataX.append(time_series[i:i+seq_length, :-1])
        dataY.append(time_series[i+seq_length, [-1]])
    return np.array(dataX), np.array(dataY)

# 하이퍼파라미터 설정
seq_length = 24
batch_size = 32

# 학습 데이터 생성
# trainX: 24시간의 피처 데이터
# trainY: 그 다음 시간에 대한 발전량 값
trainX, trainY = build_dataset(train_data[features + [target]].values, seq_length)

# 텐서로 변환
trainX_tensor = torch.tensor(trainX)
trainY_tensor = torch.tensor(trainY)

# 텐서데이터셋 생성
train_dataset = TensorDataset(trainX_tensor, trainY_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

### 학습한 시계열 모델을 사용해서 예측을 해보세요 (RNN, LSTM, Transformer, Informer 중에서 선택적으로 활용)

In [12]:
# 슬라이딩 윈도우 방식으로 2023-10-15의 24시간 발전량 예측해보기 (Direct Multi-step Forecast Strategy 혹은 Recursive Multi-step Forecast 등 이외의 방법론도 자유롭게 사용 가능)
# 필요한 라이브러리 추가
import torch.nn as nn

# RNN 모델 정의
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # RNN 레이어
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)

        # 출력 레이어
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # 초기 hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # RNN 레이어 통과
        out, _ = self.rnn(x, h0)

        # 마지막 시퀀스의 출력만 사용
        out = self.fc(out[:, -1, :])
        return out

# 하이퍼파라미터 설정
input_size = len(features)  # 입력 feature의 개수
hidden_size = 64  # RNN hidden state 크기
output_size = 1  # 발전량 예측이므로 1
num_layers = 2  # RNN 레이어 수

# 모델 생성
model = RNNModel(input_size, hidden_size, output_size, num_layers)

# GPU 사용 여부 확인 후 모델에 적용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_Y in train_loader:
        batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)

        # 모델 예측
        outputs = model(batch_X.float())
        loss = criterion(outputs, batch_Y.float())

        # 역전파 및 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크마다 손실 출력
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 테스트 데이터 준비
testX = test_data[features].values
X_test_tensor = torch.tensor(testX, dtype=torch.float32)
X_test_tensor = X_test_tensor.unsqueeze(0)

Epoch [10/100], Loss: 0.0200
Epoch [20/100], Loss: 0.0021
Epoch [30/100], Loss: 0.0006
Epoch [40/100], Loss: 0.0019
Epoch [50/100], Loss: 0.0033
Epoch [60/100], Loss: 0.0027
Epoch [70/100], Loss: 0.0013
Epoch [80/100], Loss: 0.0021
Epoch [90/100], Loss: 0.0038
Epoch [100/100], Loss: 0.0025


### 2023-10-15 발전량 예측값을 predicted_amounts에 저장하고 시각화를 실행해주세요.

In [19]:
# 예측 수행 후 predicted_amounts에 저장
model.eval()
predictions = []

# 시퀀스 데이터를 슬라이딩 윈도우 방식으로 사용하여 예측
with torch.no_grad():
    input_seq = X_test_tensor[:seq_length].unsqueeze(0)  # 24시간의 입력 시퀀스
    input_seq = input_seq.to('cuda' if torch.cuda.is_available() else 'cpu')

    for _ in range(24):  # 24시간 발전량 예측
        predicted_output = model(input_seq)
        predictions.append(predicted_output.item())

        # 슬라이딩 윈도우: 예측 결과를 다음 시퀀스의 입력으로 사용
        next_input = torch.cat((input_seq[:, 1:, :], predicted_output.unsqueeze(0).unsqueeze(2)), dim=1)
        input_seq = next_input

# 예측 결과를 scaling 복원 후 predicted_amounts로 저장
predicted_amounts = scaler_y.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()


ValueError: RNN: Expected input to be 2D or 3D, got 4D tensor instead

In [20]:
# 시각화
fig = go.Figure()

# 실제 발전량 시각화
fig.add_trace(go.Scatter(x=data['time'], y=data['amount'],
                         mode='lines', name='Actual Amount'))

# 예측된 발전량 시각화 (2023-10-15의 24시간 동안의 예측)
fig.add_trace(go.Scatter(x=test_data['time'], y=predicted_amounts,
                         mode='lines', name='Predicted Amount', line=dict(dash='dot', color='red')))

# 레이아웃 설정
fig.update_layout(title='2023-10-15 24시간 발전량 예측',
                  xaxis_title='시간',
                  yaxis_title='발전량 (kWh)')

# 그래프 출력
fig.show()

NameError: name 'predicted_amounts' is not defined

### 정답을 적어주세요.

In [ ]:
# 예측된 24시간 발전량 출력
print(f'2023-10-15 예측 발전량 (24시간): {predicted_amounts} kWh')

---

### (필수) Informer모델은 transformer 모델의 어떤 부분을 개선하고자 했나요? (차이점을 중심으로 서술)

In [ ]:
1. Self-Attention의 계산효율성
- Transformer는 시퀀스 길이가 길어질수록 계산 비용이 매우 커짐 반면에, 인포머는 중요한 쌍에만 집중하기 때문에 효율적인 계산이 가능함
2. Memory 문제 개선
- 트랜스포머는 대규모 메모리 사용이 발생하나 인포머는 불필요한 키-쿼리간 상호작용을 줄이기 때문에 메모리 사용량도 크게 줄임
3. 긴 시퀀스에서의 전반적인 정보 흐름 문제
4. 확장성

### (필수) 모델링 해석

In [ ]:
# 모델을 선택했다면 왜 선택했는지 본인만의 근거를 정리해주세요.
# 더 나아가 파라미터 선택의 기준이 있었다면 좋습니다.
코드가 안돌아가요...

### (선택) 데이터 해석

In [ ]:
# 데이터셋을 보고 느낀 생각이나 본인만의 논리 전개 방식을 정리해주세요.
# 전처리를 했다면 해당 전처리를 왜 했는지, 파생변수를 생성했다면 왜 만들었는지 등